In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import time

In [ ]:
#date_range

In [80]:
workouts = pd.read_csv('workouts.csv')

In [81]:
def epoch_end_convert(df):
    '''
    takes dataframe and converts date column to epoch milliseconds
    inputs:
        df = dataframe name
    '''
    try:
        return (int(time.mktime(time.strptime(df['end'], '%Y-%m-%d %H:%M:%S'))) - 14400)*1000
    except:
        #handles activities that have fractions of a second included
        return (int(time.mktime(time.strptime(df['end'][:19], '%Y-%m-%d %H:%M:%S'))) - 14400)*1000

In [82]:
#workouts_subset['end_epoch'] = workouts_subset.apply(epoch_end_convert, axis=1)
workouts['end_epoch'] = workouts.apply(epoch_end_convert, axis=1)

In [99]:
workouts.head()

,user_id,start,end,sport_id,name,intensity_score,z1,z2,z3,z4,z5,end_epoch
0,828,2015-09-15 20:08:00,2015-09-15 22:56:00,35,Track & Field,20.508564,1066,1810,1432,4668,679,1442357760000
1,828,2015-09-19 12:46:00,2015-09-19 14:09:00,35,Track & Field,16.823285,506,601,1861,696,22,1442671740000
2,828,2015-09-20 13:15:00,2015-09-20 16:00:00,35,Track & Field,20.506562,929,1302,1329,5353,321,1442764800000
3,828,2015-09-21 18:55:34.582,2015-09-21 18:58:37.876,1,Cycling,4.278678,8,170,5,0,0,1442861917000
4,828,2015-09-21 19:19:00,2015-09-21 21:12:00,35,Track & Field,19.176787,655,1757,1372,2781,28,1442869920000


#Adding in Time Periods (For Either Weeks or Races)

In [199]:
date_markers = pd.read_csv('epoch_dates.csv')

In [200]:
date_markers.head()

,ET_Date,date_start_epoch,race_period,bike_period,race_period_start,race_period_end,bike_period_start,bike_period_end
0,9/15/2015 0:00,1442289600000,1,1,1,0,0,0
1,9/16/2015 0:00,1442376000000,1,1,0,0,0,0
2,9/17/2015 0:00,1442462400000,1,1,0,0,0,0
3,9/18/2015 0:00,1442548800000,1,1,0,0,0,0
4,9/19/2015 0:00,1442635200000,1,1,0,0,0,0


In [201]:
date_markers = date_markers[date_markers['race_period_start'] > 0].reset_index()
#dictionary of race period to start epoch
race_start_dict = {}
for i in range(0, len(date_markers)):
    race_start_dict[date_markers.values[i][3]] = date_markers.values[i][2]

In [202]:
race_start_dict

{1L: 1442289600000L,
 2L: 1443240000000L,
 3L: 1445054400000L,
 4L: 1446264000000L,
 5L: 1447473600000L}

In [236]:
workouts_subset = workouts[:30]
workouts_subset.head(10)

,user_id,start,end,sport_id,name,intensity_score,z1,z2,z3,z4,z5,end_epoch
0,828,2015-09-15 20:08:00,2015-09-15 22:56:00,35,Track & Field,20.508564,1066,1810,1432,4668,679,1442357760000
1,828,2015-09-19 12:46:00,2015-09-19 14:09:00,35,Track & Field,16.823285,506,601,1861,696,22,1442671740000
2,828,2015-09-20 13:15:00,2015-09-20 16:00:00,35,Track & Field,20.506562,929,1302,1329,5353,321,1442764800000
3,828,2015-09-21 18:55:34.582,2015-09-21 18:58:37.876,1,Cycling,4.278678,8,170,5,0,0,1442861917000
4,828,2015-09-21 19:19:00,2015-09-21 21:12:00,35,Track & Field,19.176787,655,1757,1372,2781,28,1442869920000
5,828,2015-09-22 20:31:00,2015-09-22 22:00:00,35,Track & Field,18.825382,767,992,1291,806,1323,1442959200000
6,828,2015-09-23 19:31:48.525,2015-09-23 20:21:16.705,35,Track & Field,16.790126,126,499,703,1452,89,1443039676000
7,828,2015-09-23 20:47:48.968,2015-09-23 20:58:34.201,44,Yoga,4.081088,142,13,0,0,0,1443041914000
8,828,2015-09-25 18:30:00,2015-09-25 20:47:00,0,Running,19.797324,1453,1601,1803,2406,630,1443214020000
9,828,2015-09-26 13:51:00,2015-09-26 15:00:00,0,Running,18.177086,538,353,557,2124,365,1443279600000


In [237]:
workouts_subset['race_period'] = 0

C:\Users\BUCKBEAK\Anaconda\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [238]:
race_start_dict[1]

1442289600000L

In [239]:
def race(df):
    if (df.end_epoch >= race_start_dict[1]) and (df.end_epoch < race_start_dict[2]):
        return 1
    else if (df.end_epoch >= race_start_dict[2]) and (df.end_epoch < race_start_dict[3]):
        return 2
    else if (df.end_epoch >= race_start_dict[3]) and (df.end_epoch < race_start_dict[4]):
        return 3
    else if (df.end_epoch >= race_start_dict[4]) and (df.end_epoch < race_start_dict[5]):
        return 4
    else if (df.end_epoch >= race_start_dict[5]) and (df.end_epoch < 1448164800000):
        return 5


In [240]:
workouts_subset['race_period'] = workouts_subset.apply(race, axis=1)

C:\Users\BUCKBEAK\Anaconda\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [241]:
workouts_subset

,user_id,start,end,sport_id,name,intensity_score,z1,z2,z3,z4,z5,end_epoch,race_period
0,828,2015-09-15 20:08:00,2015-09-15 22:56:00,35,Track & Field,20.508564,1066,1810,1432,4668,679,1442357760000,1
1,828,2015-09-19 12:46:00,2015-09-19 14:09:00,35,Track & Field,16.823285,506,601,1861,696,22,1442671740000,1
2,828,2015-09-20 13:15:00,2015-09-20 16:00:00,35,Track & Field,20.506562,929,1302,1329,5353,321,1442764800000,1
3,828,2015-09-21 18:55:34.582,2015-09-21 18:58:37.876,1,Cycling,4.278678,8,170,5,0,0,1442861917000,1
4,828,2015-09-21 19:19:00,2015-09-21 21:12:00,35,Track & Field,19.176787,655,1757,1372,2781,28,1442869920000,1
5,828,2015-09-22 20:31:00,2015-09-22 22:00:00,35,Track & Field,18.825382,767,992,1291,806,1323,1442959200000,1
6,828,2015-09-23 19:31:48.525,2015-09-23 20:21:16.705,35,Track & Field,16.790126,126,499,703,1452,89,1443039676000,1
7,828,2015-09-23 20:47:48.968,2015-09-23 20:58:34.201,44,Yoga,4.081088,142,13,0,0,0,1443041914000,1
8,828,2015-09-25 18:30:00,2015-09-25 20:47:00,0,Running,19.797324,1453,1601,1803,2406,630,1443214020000,1
9,828,2015-09-26 13:51:00,2015-09-26 15:00:00,0,Running,18.177086,538,353,557,2124,365,1443279600000,NaN


In [ ]:
def add_periods(date_markers, df, buildup_days=0, version):
    '''
    Adds race periods to the input dataframe 'df'
        Inputs: 
            date_markers: dataframe with the ET date, the start epoch time of that date, and which race period the date is in
            df: the dataframe to add these date markers to
            buildup_days: if 0, we use the entire time period before the race/bike, after the previous one
                if not 0, and instead a value "n", we look at the "n" days before the race/bike as the "period"
            version: takes two values: "bike", or "race", and treats appropriately
    '''
    if buildup_days=0:
        if version=="race":
            #Collapse the data frame into only the start and end dates for each race period
            date_markers = date_markers[date_markers['race_period_start'] > 0].reset_index()
            #dictionary of race period to start epoch
            race_start_dict = {}
            for i in range(0, len(date_markers)):
            race_start_dict[date_markers.values[i][3]] = date_markers.values[i][2]
            
            #Now label each race period as 1, 2, 3, 4, or 5
            